In [ ]:
### Xi -> reference representation  Xj -> target representation
## adjust reference Xi  to  Xi' => CKA(Xi', Xj) ----> 1
# R -> min ||XjR - Xi'||

##### adjust : keep similar samples and highlight the diff samples' difference 
# step1: find diff samples and same samples
# step2: adjust diff samples 

In [1]:
#### 

# REF_PATH : reference dataset path
# CONFUSION_PATH : benchmark1
# EXCHANGE_PATH : benchmark2

import torch
import sys
sys.path.append("..")
from singleVis.data import NormalDataProvider



REF_PATH = "/home/yifan/dataset/clean_ref/pairflip/cifar10/0"


CLEAN_PATH = "/home/yifan/dataset/clean/pairflip/cifar10/0"

sys.path.append(REF_PATH)


from config import config

SETTING = config["SETTING"]
CLASSES = config["CLASSES"]
DATASET = config["DATASET"]
GPU_ID = config["GPU"]
EPOCH_START = config["EPOCH_START"]
EPOCH_END = config["EPOCH_END"]
EPOCH_PERIOD = config["EPOCH_PERIOD"]

# Training parameter (subject model)
TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
LEN = TRAINING_PARAMETER["train_num"]

# Training parameter (visualization model)
VISUALIZATION_PARAMETER = config["VISUALIZATION"]
PREPROCESS = VISUALIZATION_PARAMETER["PREPROCESS"]
LAMBDA = VISUALIZATION_PARAMETER["LAMBDA"]
B_N_EPOCHS = VISUALIZATION_PARAMETER["BOUNDARY"]["B_N_EPOCHS"]
L_BOUND = VISUALIZATION_PARAMETER["BOUNDARY"]["L_BOUND"]
INIT_NUM = VISUALIZATION_PARAMETER["INIT_NUM"]
ALPHA = VISUALIZATION_PARAMETER["ALPHA"]
BETA = VISUALIZATION_PARAMETER["BETA"]
MAX_HAUSDORFF = VISUALIZATION_PARAMETER["MAX_HAUSDORFF"]
# HIDDEN_LAYER = VISUALIZATION_PARAMETER["HIDDEN_LAYER"]
ENCODER_DIMS = VISUALIZATION_PARAMETER["ENCODER_DIMS"]
DECODER_DIMS = VISUALIZATION_PARAMETER["DECODER_DIMS"]
S_N_EPOCHS = VISUALIZATION_PARAMETER["S_N_EPOCHS"]
T_N_EPOCHS = VISUALIZATION_PARAMETER["T_N_EPOCHS"]
N_NEIGHBORS = VISUALIZATION_PARAMETER["N_NEIGHBORS"]
PATIENT = VISUALIZATION_PARAMETER["PATIENT"]
MAX_EPOCH = VISUALIZATION_PARAMETER["MAX_EPOCH"]

VIS_MODEL_NAME = 'vis'
EVALUATION_NAME = VISUALIZATION_PARAMETER["EVALUATION_NAME"]

SEGMENTS = [(EPOCH_START, EPOCH_END)]
# define hyperparameters
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")

import Model.model as subject_model
net = eval("subject_model.{}()".format(NET))


ref_provider = NormalDataProvider(REF_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, split=-1, device=DEVICE, classes=CLASSES,verbose=1)


clean_provider = NormalDataProvider(CLEAN_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, split=-1, device=DEVICE, classes=CLASSES,verbose=1)


/home/yifan/miniconda3/envs/deepdebugger/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Finish initialization...
Finish initialization...


In [2]:
ref_train_data = ref_provider.train_representation(200).squeeze()
clean_train_data = clean_provider.train_representation(200).squeeze()

In [4]:
ref_prediction = ref_provider.get_pred(200, ref_provider.train_representation(200))
clean_prediction = clean_provider.get_pred(200, clean_provider.train_representation(200))

100%|██████████| 250/250 [00:00<00:00, 9434.90it/s]


In [15]:
ref_prediction_res = ref_provider.get_pred(200, ref_provider.train_representation(200)).argmax(1)

clean_prediction_res = clean_provider.get_pred(200, clean_provider.train_representation(200)).argmax(1)

diff_indicates = []
same_indicates = []
pairs = []
for i in range(len(ref_prediction)):
    if clean_prediction_res[i] != ref_prediction_res[i]:
        
        diff_indicates.append(i)
        pairs.append({"ref predict": ref_prediction_res[i], "target prediction": clean_prediction_res[i]})
    else:
        same_indicates.append(i)
print('diff distance: diff_num/ total_num:',len(diff_indicates)/50000)
# print('diff_indicated:',diff_indicates, pairs)

100%|██████████| 250/250 [00:00<00:00, 9293.74it/s]

diff distance: diff_num/ total_num: 0.00034


In [21]:
from scipy.special import softmax
import numpy as np
def get_conf(pred):
    scores = np.amax(softmax(pred, axis=1), axis=1)
    return scores
# def filter_conf(pred, index):
#     scores = np.amax(softmax(pred, axis=1), axis=1)
#     res = np.argwhere(np.logical_and(scores<=1, scores>=0)).squeeze().tolist()
#     return res[index]

In [22]:
ref_scores = get_conf(ref_prediction)
clean_scores =  get_conf(clean_prediction)

In [23]:
ref_scores

array([0.9999695, 1.       , 1.       , ..., 1.       , 1.       ,
       0.9999543], dtype=float32)

In [24]:
clean_scores

array([1.        , 1.        , 1.        , ..., 1.        , 1.        ,
       0.99967575], dtype=float32)

In [30]:
diff_indicates = []
same_indicates = []
for i in range(len(ref_prediction)):
    if clean_prediction_res[i] == ref_prediction_res[i] and ref_scores[i] == clean_scores[i]:
        
        same_indicates.append(i)
        pairs.append({"ref predict": ref_prediction_res[i], "target prediction": clean_prediction_res[i]})
    else:
        diff_indicates.append(i)

In [31]:
len(diff_indicates)

22283

In [32]:
import numpy as np
from sklearn.cross_decomposition import CCA
# 建立模型
X = ref_prediction[same_indicates]
Y = clean_prediction[same_indicates]
cca = CCA(n_components=1)
# 训练数据
cca.fit(X, Y)
# print(X)
X_train_r, Y_train_r = cca.transform(X, Y)
# print(X_train_r)
print('referece epoch 200 vs current epoch 200')
print(np.corrcoef(X_train_r[:, 0], Y_train_r[:, 0])[0, 1]) #输出相关系数

referece epoch 200 vs current epoch 200
0.9604035124077683


In [18]:
import numpy as np
from sklearn.cross_decomposition import CCA
# 建立模型
X = ref_prediction
Y = clean_prediction
cca = CCA(n_components=1)
# 训练数据
cca.fit(X, Y)
# print(X)
X_train_r, Y_train_r = cca.transform(X, Y)
# print(X_train_r)
print('referece epoch 200 vs current epoch 200')
print(np.corrcoef(X_train_r[:, 0], Y_train_r[:, 0])[0, 1]) #输出相关系数

referece epoch 200 vs current epoch 200
0.9576635042671429
